In [25]:
#import sys
#print(sys.executable)

In [26]:
#/Users/mei/Documents/project

In [27]:
import spacy
import pandas as pd
import glob
import os
from bs4 import BeautifulSoup

import seaborn as sns
sns.set_style('darkgrid')

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
nltk.download('punkt')

from wordcloud import WordCloud


import re, string, random

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

#setting pandas display options
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1) 

[nltk_data] Downloading package punkt to /Users/mei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-27-6e327f50ed7d>:31: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Functions + Helper variables

In [28]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [29]:
# removes keywords from your text data

def map_keyword(df, kw, kw_list, col = 'title', col_name = 'title_preprocessed'):
    
    # filter to only replace keyword data
    temp = df.loc[df['keyword'] == kw]
    
    # replace keyword in string column
    temp[col_name] = temp[col].str.replace('|'.join(kw_list), 'keyword') 
    
    return(temp)

In [30]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        
        expanded_contraction = first_char + expanded_contraction[1:]
        
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    
    return expanded_text

In [31]:
def remove_stopwords(tokens, is_lower_case = True):
    
    #tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    
    return filtered_text

In [32]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [33]:
# Define a function to plot word cloud
def plot_cloud(wordcloud):
    
    # Set figure size
    plt.figure(figsize = (40, 30))
    
    # Display image
    plt.imshow(wordcloud) 
    
    # No axis details
    plt.axis("off");

## Load in files

In [75]:
comments_df = pd.read_csv('MEGA_ffa_comments_identity.csv')

In [56]:
comments_df.head(n = 3)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword
0,fuqh75f,t3_gzpwr7,"Hello, I’m a fashion designer based in CT. Born In AL. Raised in NYC.\n\n[Here Is All My Social Accounts; Website; And My Most Recent Video On Me Making My Koi Fish Dress](https://linktr.ee/derells.style) \n\n✨ Style Is Something Each Of Us Already Has, All We Need To Do Is Find It! ✨\n\n🖤 We hope to see you around 🖤",t3_gzpwr7,BIPOC
1,fvfycp8,t3_gzpwr7,"Highly recommend following [Babbarra Designs](https://www.instagram.com/babbarradesigns/). It's an Australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. It's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC
2,g34oia4,t3_gzpwr7,[Birr Apparel ](www.birrapparel.com),t3_gzpwr7,BIPOC


## Lowercasing

In [76]:
comments_df['comment_body'] = comments_df['comment_body'].str.lower()

In [58]:
comments_df.head(n = 2)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword
0,fuqh75f,t3_gzpwr7,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤",t3_gzpwr7,BIPOC
1,fvfycp8,t3_gzpwr7,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it's an australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC


## Expand contractions

In [77]:
comments_df['comment_body'] = comments_df['comment_body'].apply(str)

In [78]:
comments_df['comment_body_preprocessed'] = comments_df['comment_body'].apply(expand_contractions)

In [79]:
comments_df.head(n = 6)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword,comment_body_preprocessed
0,fuqh75f,t3_gzpwr7,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤",t3_gzpwr7,BIPOC,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤"
1,fvfycp8,t3_gzpwr7,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it's an australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it is an australian aboriginal womens center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it is also just a really joyous page with lots of images of the artists showing off their own designs or garments."
2,g34oia4,t3_gzpwr7,[birr apparel ](www.birrapparel.com),t3_gzpwr7,BIPOC,[birr apparel ](www.birrapparel.com)
3,fv27w43,t3_gzpwr7,[deleted],t3_gzpwr7,BIPOC,[deleted]
4,ftp0gyl,t3_gzpwr7,[zelie for she](https://www.instagram.com/zelieforshe/) does plus sized basics. based out of la.,t3_gzpwr7,BIPOC,[zelie for she](https://www.instagram.com/zelieforshe/) does plus sized basics. based out of la.
5,g3lgz6a,t3_gzpwr7,"if you're based somewhere in asia and really like cute summer tops that use a lot of upcycled and/or organic materials, my girlfriend started her own brand. \n\nwe're an asian couple based in singapore and we try to use handmade fabrics sourced ethically from india! \n\n[cryafty](https://www.instagram.com/cryafty/)",t3_gzpwr7,BIPOC,"if you are based somewhere in asia and really like cute summer tops that use a lot of upcycled and/or organic materials, my girlfriend started her own brand. \n\nwe are an asian couple based in singapore and we try to use handmade fabrics sourced ethically from india! \n\n[cryafty](https://www.instagram.com/cryafty/)"


## Removing punctuation and noise

In [81]:
# remove unnecessary punctuation
tokenizer = nltk.RegexpTokenizer(r"\w+")
comments_df['comment_body_preprocessed'] = comments_df['comment_body_preprocessed'].apply(tokenizer.tokenize)

In [82]:
# add them back into a long string
comments_df['comment_body_preprocessed'] = comments_df['comment_body_preprocessed'].str.join(" ")

In [83]:
comments_df.head(n = 2)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword,comment_body_preprocessed
0,fuqh75f,t3_gzpwr7,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤",t3_gzpwr7,BIPOC,hello i m a fashion designer based in ct born in al raised in nyc here is all my social accounts website and my most recent video on me making my koi fish dress https linktr ee derells style style is something each of us already has all we need to do is find it we hope to see you around
1,fvfycp8,t3_gzpwr7,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it's an australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC,highly recommend following babbarra designs https www instagram com babbarradesigns it is an australian aboriginal womens center that produces fashion textiles and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles it is also just a really joyous page with lots of images of the artists showing off their own designs or garments


## Tokenization

In [84]:
comments_df['tokenized'] = comments_df['comment_body_preprocessed'].apply(word_tokenize)

In [85]:
comments_df.head(n = 2)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword,comment_body_preprocessed,tokenized
0,fuqh75f,t3_gzpwr7,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤",t3_gzpwr7,BIPOC,hello i m a fashion designer based in ct born in al raised in nyc here is all my social accounts website and my most recent video on me making my koi fish dress https linktr ee derells style style is something each of us already has all we need to do is find it we hope to see you around,"[hello, i, m, a, fashion, designer, based, in, ct, born, in, al, raised, in, nyc, here, is, all, my, social, accounts, website, and, my, most, recent, video, on, me, making, my, koi, fish, dress, https, linktr, ee, derells, style, style, is, something, each, of, us, already, has, all, we, need, to, do, is, find, it, we, hope, to, see, you, around]"
1,fvfycp8,t3_gzpwr7,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it's an australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC,highly recommend following babbarra designs https www instagram com babbarradesigns it is an australian aboriginal womens center that produces fashion textiles and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles it is also just a really joyous page with lots of images of the artists showing off their own designs or garments,"[highly, recommend, following, babbarra, designs, https, www, instagram, com, babbarradesigns, it, is, an, australian, aboriginal, womens, center, that, produces, fashion, textiles, and, art, and, they, showcase, the, artists, and, designers, and, the, skill, that, goes, into, making, the, breathtaking, textiles, it, is, also, just, a, really, joyous, page, with, lots, of, images, of, the, artists, showing, off, their, own, designs, or, garments]"


## Stop-word removal

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
stopword_list = stopwords.words('english')

In [86]:
comments_df['comment_body_preprocessed'] = comments_df['tokenized'].apply(remove_stopwords)

In [87]:
comments_df.head(n = 2)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword,comment_body_preprocessed,tokenized
0,fuqh75f,t3_gzpwr7,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤",t3_gzpwr7,BIPOC,hello fashion designer based ct born al raised nyc social accounts website recent video making koi fish dress https linktr ee derells style style something us already need find hope see around,"[hello, i, m, a, fashion, designer, based, in, ct, born, in, al, raised, in, nyc, here, is, all, my, social, accounts, website, and, my, most, recent, video, on, me, making, my, koi, fish, dress, https, linktr, ee, derells, style, style, is, something, each, of, us, already, has, all, we, need, to, do, is, find, it, we, hope, to, see, you, around]"
1,fvfycp8,t3_gzpwr7,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it's an australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC,highly recommend following babbarra designs https www instagram com babbarradesigns australian aboriginal womens center produces fashion textiles art showcase artists designers skill goes making breathtaking textiles also really joyous page lots images artists showing designs garments,"[highly, recommend, following, babbarra, designs, https, www, instagram, com, babbarradesigns, it, is, an, australian, aboriginal, womens, center, that, produces, fashion, textiles, and, art, and, they, showcase, the, artists, and, designers, and, the, skill, that, goes, into, making, the, breathtaking, textiles, it, is, also, just, a, really, joyous, page, with, lots, of, images, of, the, artists, showing, off, their, own, designs, or, garments]"


## Stemming

In [88]:
def simple_stemmer(text):
    
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    
    return text

In [89]:
comments_df['stemmed'] = comments_df['comment_body_preprocessed'].apply(simple_stemmer)

In [90]:
comments_df.head(n = 2)

,comment_id,comment_parent_id,comment_body,comment_link_id,keyword,comment_body_preprocessed,tokenized,stemmed
0,fuqh75f,t3_gzpwr7,"hello, i’m a fashion designer based in ct. born in al. raised in nyc.\n\n[here is all my social accounts; website; and my most recent video on me making my koi fish dress](https://linktr.ee/derells.style) \n\n✨ style is something each of us already has, all we need to do is find it! ✨\n\n🖤 we hope to see you around 🖤",t3_gzpwr7,BIPOC,hello fashion designer based ct born al raised nyc social accounts website recent video making koi fish dress https linktr ee derells style style something us already need find hope see around,"[hello, i, m, a, fashion, designer, based, in, ct, born, in, al, raised, in, nyc, here, is, all, my, social, accounts, website, and, my, most, recent, video, on, me, making, my, koi, fish, dress, https, linktr, ee, derells, style, style, is, something, each, of, us, already, has, all, we, need, to, do, is, find, it, we, hope, to, see, you, around]",hello fashion design base ct born al rais nyc social account websit recent video make koi fish dress http linktr ee derel style style someth us alreadi need find hope see around
1,fvfycp8,t3_gzpwr7,"highly recommend following [babbarra designs](https://www.instagram.com/babbarradesigns/). it's an australian aboriginal women's center that produces fashion, textiles, and art and they showcase the artists and designers and the skill that goes into making the breathtaking textiles. it's also just a really joyous page with lots of images of the artists showing off their own designs or garments.",t3_gzpwr7,BIPOC,highly recommend following babbarra designs https www instagram com babbarradesigns australian aboriginal womens center produces fashion textiles art showcase artists designers skill goes making breathtaking textiles also really joyous page lots images artists showing designs garments,"[highly, recommend, following, babbarra, designs, https, www, instagram, com, babbarradesigns, it, is, an, australian, aboriginal, womens, center, that, produces, fashion, textiles, and, art, and, they, showcase, the, artists, and, designers, and, the, skill, that, goes, into, making, the, breathtaking, textiles, it, is, also, just, a, really, joyous, page, with, lots, of, images, of, the, artists, showing, off, their, own, designs, or, garments]",highli recommend follow babbarra design http www instagram com babbarradesign australian aborigin women center produc fashion textil art showcas artist design skill goe make breathtak textil also realli joyou page lot imag artist show design garment


In [53]:
#comments_df.to_csv(r'/Users/mei/Documents/project/MEGA_ffa_comments_identity_preprocessed.csv',index = False, header=True)